## facial point 정보를 수집하여 csv로 저장

* image-url로 부터 facial point 수집하는 api

In [27]:
import urllib.parse
import urllib.request
import json

## face detect api call 
def face_detect(imgurl):
    url = 'http://apius.faceplusplus.com/v2/detection/detect'
    headers = { 'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Origin':'http://www.faceplusplus.com', 
               'Accept-Language':'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4', 
               'Content-type':'application/x-www-form-urlencoded', 
               'Accept':'*/*', 
               'Referer':'http://www.faceplusplus.com/demo-landmark/', 
               'Connection':'keep-alive'
              }
    values = {'attribute' : 'none',
              'url' : imgurl,
              'api_key' : 'e6f2c06ac2b93b1ae6aad80b90d49eb0',
              'api_secret' : '8o_amgigF4dV4EHZ2OOVEejDeXq4qFab'}

    data = urllib.parse.urlencode(values)
    data = data.encode(u'utf-8')
    req = urllib.request.Request(url, data, headers)
    response = urllib.request.urlopen(req)
    response_data = response.read()
    response.close()
    return json.loads(response_data.decode('utf-8'))

## face landmark api call by face_id
def face_landmark(faceid):
    url = 'http://apius.faceplusplus.com/v2/detection/landmark'
    headers = { 'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Origin':'http://www.faceplusplus.com', 
               'Accept-Language':'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4', 
               'Content-type':'application/x-www-form-urlencoded', 
               'Accept':'*/*', 
               'Referer':'http://www.faceplusplus.com/demo-landmark/', 
               'Connection':'keep-alive'
              }
    values = {'face_id' : faceid,
              'type' : '83p',
              'api_key' : 'e6f2c06ac2b93b1ae6aad80b90d49eb0',
              'api_secret' : '8o_amgigF4dV4EHZ2OOVEejDeXq4qFab'}
    data = urllib.parse.urlencode(values)
    data = data.encode(u'utf-8')

    req = urllib.request.Request(url, data, headers)

    response = urllib.request.urlopen(req)
    response_data = response.read()
    response.close()
    return json.loads(response_data.decode('utf-8'))

def  extract_face_landmark( imgurl, face_index=0 ) :
    fd = face_detect(imgurl)
    face_id = fd['face'][face_index]['face_id']
    return face_landmark(face_id)
    

In [28]:
extract_face_landmark('https://fbcdn-sphotos-g-a.akamaihd.net/hphotos-ak-xta1/v/t1.0-9/12122398_906503296086905_9053689686321602818_n.jpg?oh=80fcaf06feebc607119139261daa82ee&oe=5696C59E&__gda__=1452163095_656cdb05f7f03f0920879b17b79041be' )

{'result': [{'face_id': 'becacf4783227833bd4348dd459863cd',
   'landmark': {'contour_chin': {'x': 46.870333, 'y': 39.516444},
    'contour_left1': {'x': 39.563, 'y': 24.433778},
    'contour_left2': {'x': 39.6425, 'y': 26.722},
    'contour_left3': {'x': 39.836667, 'y': 28.721333},
    'contour_left4': {'x': 40.219, 'y': 30.990889},
    'contour_left5': {'x': 40.659333, 'y': 32.897778},
    'contour_left6': {'x': 41.447333, 'y': 34.854222},
    'contour_left7': {'x': 42.452333, 'y': 36.781556},
    'contour_left8': {'x': 43.545833, 'y': 38.136667},
    'contour_left9': {'x': 45.202167, 'y': 39.372444},
    'contour_right1': {'x': 54.509167, 'y': 22.996222},
    'contour_right2': {'x': 54.733833, 'y': 25.442667},
    'contour_right3': {'x': 54.652167, 'y': 27.907778},
    'contour_right4': {'x': 54.464167, 'y': 30.252222},
    'contour_right5': {'x': 53.924167, 'y': 32.507333},
    'contour_right6': {'x': 53.326167, 'y': 34.674667},
    'contour_right7': {'x': 52.0905, 'y': 36.589111},


In [94]:
import json
import numpy as np
from scipy.spatial import distance

def pos_distance(l,r):
#    print(l,r)
    return round(distance.euclidean( [l['x'], l['y']], [r['x'], r['y']] ),5)

def pos_center(l,r):
    return {'x':(l['x']+r['x'])/2, 'y':(l['y']+r['y'])/2}

def extract_face_feature(facial_info):
    cal_items = {'face_width'    :['contour_left9', 'contour_right9']
                 ,'face_height'  :['eyebrow_center','contour_chin']
                 ,'mouth_width'  :['mouth_left_corner','mouth_right_corner']  
                 ,'nose_width'   :['nose_left','nose_right']  
                 ,'eye_width'    :['left_eye_left_corner','right_eye_right_corner']  
                 ,'eye_to_eye'   :['left_eye_center','right_eye_center']  
                 ,'eyebrow_width':['left_eyebrow_upper_middle','left_eyebrow_lower_middle']  
                 ,'lip_to_chin'  :['mouth_lower_lip_bottom','contour_chin']  
                 ,'lip_to_nose'  :['nose_tip','mouth_upper_lip_top']}
    
    keypoint = {}
    
    ## json string to object
    info = facial_info
    if type(facial_info) == str:
        info = json.loads(facial_info)
    
    facial_pos = info['result'][0]['landmark']
    ## add center position
    facial_pos['eyebrow_center'] = pos_center( facial_pos['left_eyebrow_upper_middle'], facial_pos['right_eyebrow_upper_middle']) 
    
    ## calculate key point
    for (k,v) in cal_items.items(): ## iteritems() for python2.7
        keypoint[k] = pos_distance(facial_pos[v[0]], facial_pos[v[1]])
        
    return keypoint

def normalize_dict(data, max_value):
    for k in data.keys():
        data[k] = round(data[k]/max_value,5)
    return data

def get_feature_vector( keypoint ):
    ff = extract_face_feature(keypoint)
    ff = normalize_dict(ff, ff['face_height'])
    return ff

In [ ]:
def extract_face_feature(facial_info):
    cal_items = {'face_width'    :['contour_left9', 'contour_right9']
                 ,'face_height'  :['eyebrow_center','contour_chin']
                 ,'mouth_width'  :['mouth_left_corner','mouth_right_corner']  
                 ,'nose_width'   :['nose_left','nose_right']  
                 ,'eye_width'    :['left_eye_left_corner','right_eye_right_corner']  
                 ,'eye_to_eye'   :['left_eye_center','right_eye_center']  
                 ,'eyebrow_width':['left_eyebrow_upper_middle','left_eyebrow_lower_middle']  
                 ,'lip_to_chin'  :['mouth_lower_lip_bottom','contour_chin']  
                 ,'lip_to_nose'  :['nose_tip','mouth_upper_lip_top']}
    
    keypoint = {}
    
    ## json string to object
    info = facial_info
    if type(facial_info) == str:
        info = json.loads(facial_info)
    
    facial_pos = info['result'][0]['landmark']
    ## add center position
    facial_pos['eyebrow_center'] = pos_center( facial_pos['left_eyebrow_upper_middle'], facial_pos['right_eyebrow_upper_middle']) 
    
    ## calculate key point
    for (k,v) in cal_items.items(): ## iteritems() for python2.7
        keypoint[k] = pos_distance(facial_pos[v[0]], facial_pos[v[1]])
        
    return keypoint


In [102]:
## 수집할 image url 
image_data = { '최규민1':'https://fbcdn-sphotos-g-a.akamaihd.net/hphotos-ak-xta1/v/t1.0-9/12122398_906503296086905_9053689686321602818_n.jpg?oh=80fcaf06feebc607119139261daa82ee&oe=5696C59E&__gda__=1452163095_656cdb05f7f03f0920879b17b79041be'
              , '최규민2' : 'https://fbcdn-sphotos-e-a.akamaihd.net/hphotos-ak-xpa1/t31.0-8/1517851_906503269420241_4763040186730610966_o.jpg'
              , '최규민3' : 'https://fbcdn-sphotos-b-a.akamaihd.net/hphotos-ak-xtp1/v/t1.0-9/12118701_906503272753574_7177447444403669300_n.jpg?oh=3c594eb747cee0fad0417f661953779e&oe=56924967&__gda__=1456109393_d48c1b3f84d74dd8dbcaac4d31e12429'
              , '와이프1' : 'https://fbcdn-sphotos-a-a.akamaihd.net/hphotos-ak-xpa1/v/t1.0-9/12108061_907705495966685_5481740761293586247_n.jpg?oh=398bdb849e0b91c4bb177ba84989302f&oe=56D21979&__gda__=1451609793_60d987ad6bc86213787e0530858396e1'
              , '아기1' : 'https://scontent.xx.fbcdn.net/hphotos-xpt1/v/t1.0-9/12074727_907707092633192_9020724559277292808_n.jpg?oh=5759280c32c56e99159d974a13f44bcb&oe=569D6AFA'
              , '전명훈1' : 'https://fbcdn-sphotos-h-a.akamaihd.net/hphotos-ak-xap1/v/t1.0-9/12140604_972525119474378_7475998966124883876_n.jpg?oh=383f560d042e194e9152c5919286fab8&oe=56CE8322&__gda__=1456399102_e2ac965c1f3878d6a66c966ec888e45a'
              , '전명훈2' : 'https://fbcdn-sphotos-f-a.akamaihd.net/hphotos-ak-xap1/v/t1.0-9/12122612_972525132807710_7394758924527322556_n.jpg?oh=e422202838f486fdc2bb27941abb46a0&oe=568728E9&__gda__=1453230645_58f4314b4560b0a89a48287f87e31ebc'
              , '양정길1' : 'https://fbcdn-sphotos-d-a.akamaihd.net/hphotos-ak-xpl1/v/t1.0-9/12096634_420268864849369_4650861756174901140_n.jpg?oh=3d043deaa1d94371c2889987e90b31fe&oe=569E8384&__gda__=1452120840_77764404987b26282bcbed4d93470751'
              , '양정길2' : 'https://fbcdn-sphotos-f-a.akamaihd.net/hphotos-ak-xap1/v/t1.0-9/12074518_420268868182702_5807506611890573511_n.jpg?oh=3c43df8daa5b2e6cc13d30bef1bb976f&oe=568D00F3&__gda__=1453294639_f29144c19bc0b56a19f29d74d9ba3c79'
              , '양정길3' : 'https://fbcdn-sphotos-h-a.akamaihd.net/hphotos-ak-xpt1/v/t1.0-9/12107776_420268898182699_873741782931745747_n.jpg?oh=89833ac3449bbb44fbfa60c47f0bb0a1&oe=5684CA0C&__gda__=1456380231_e9088b09832600a1eade233900ccb068'
              , '원빈' : 'http://image.fmkorea.com/files/attach/new/20141213/486616/85599953/86802006/da08e25010e6001bf565cd5cca3ca359.jpg'
              , '김수현' : 'http://cfile10.uf.tistory.com/image/167718454EB74A8C07D3DA'
             }


In [103]:
landmark_dict = {}
for name, img_url in image_data.items(): 
    try:
        keypoint = extract_face_landmark(img_url)
        landmark_dict[name] = get_feature_vector(keypoint)
    except:
        print ('extract fail : {0}'.format(name))

In [104]:
## dict to dataframe
landmark_ds = pd.DataFrame.from_dict( landmark_dict, orient='index')

In [105]:
## write to csv file
landmark_ds.to_csv('/Users/goodvc/data/fastcampus/week2/resource/landmark.csv')

In [106]:
landmark_ds = pd.DataFrame.from_csv('/Users/goodvc/data/fastcampus/week2/resource/landmark.csv')